## Loading libs

In [37]:
# File handling libs
import pandas as pd
import pprint
import glob
import csv
from os import listdir
from os.path import isfile, join, split

# Timeseries data
import datetime
from pandas.tseries.frequencies import to_offset

# Factor plot
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 10]
%matplotlib inline

# Linear Resampling irregular timeseries data
def resample(df, rate='T', short_rate='S', max_gap=None):
    """ Resample (unevenly spaced) timeseries data linearly by first upsampling to a
        high frequency (short_rate) then downsampling to the desired rate.
    
    :param df: a pandas timeseries object
    :param rate: rate that df should be resampled to
    :param short_rate: intermediate upsampling rate; if None, smallest interval of df is used
    :param max_gap: null intervals larger than `max_gap` are being treated as missing
        data and not interpolated. if None, always interpolate. must be provided as pandas
        frequency string format, e.g. '6h'
    """
    plt.rcParams['figure.figsize'] = [15, 10]
    
    # Resetting index to elapsed time 7200s
    # Setting first value to 0
    df.loc[-1] = df.loc[0]
    df.index = df.index + 1 
    df = df.sort_index() 
    df.loc[0,'elapsed_time'] = 0
    df.loc[0,'n_trial'] = 0

    # Setting last value to 7200
    df = df.loc[df['elapsed_time'] <= 7200]
    df = df.append(df.iloc[-1]).reset_index(drop=True)
    df.iloc[-1, df.columns.get_loc('elapsed_time')] = 7200
    df.iloc[-1, df.columns.get_loc('n_trial')] = df.iloc[-2, df.columns.get_loc('n_trial')]+1

    # Converting to timedelta and setting time as index
    df['elapsed_time'] = df['elapsed_time'].apply(lambda x : datetime.timedelta(seconds=x))
    df = df.set_index('elapsed_time')
    
    # return series if empty
    if df.empty:
        return df

    # check for timedelta index
    assert isinstance(
        df.index[0], pd.Timedelta), 'Object must have a datetime-like index.'

    # sort df by time
    df.sort_index(inplace=True)

    # create timedelta from frequency string
    rate_delta = to_offset(rate).delta

    # compute time intervals
    diff = np.diff(df.index) / np.timedelta64(1, 's')

    if max_gap is not None:
        # identify intervals in df larger than max_gap
        idx = np.where(np.greater(diff, to_offset(max_gap).delta.total_seconds()))[0]
        start = df.index[idx].tolist()
        stop = df.index[idx + 1].tolist()
        # store start and stop indices of large intervals
        big_gaps = list(zip(start, stop))

    if short_rate is None:
        # use minimal nonzero interval of original series as short_rate
        short_rate = '%dS' % diff[np.nonzero(diff)].min()
        # create timedelta from frequency string
        short_rate_delta = to_offset(short_rate).delta
        # if smallest interval is still larger than rate, use rate instead
        if short_rate_delta > rate_delta:
            short_rate = rate
    else:
        # convert frequency string to timedelta
        short_rate_delta = to_offset(short_rate).delta
        # make sure entered short_rate is smaller than rate
        assert rate_delta >= short_rate_delta, 'short_rate must be <= rate'

    # upsample to short_rate
    df = df.resample(short_rate).mean().interpolate()

    # downsample to desired rate
    df = df.resample(rate).ffill()

    # replace values in large gap itervals with NaN
    if max_gap is not None:
        for start, stop in big_gaps:
            df[start:stop] = None
    
    return df


def get_avg_losses():
    # Create df of full fps using glob to return all csv files under final dir
    files = pd.DataFrame([file for file in glob.glob("final/*/*.csv")], columns=["fullpath"])
    
    print(len(files))
    #    fullpath
    # 0  final\run_1\*.csv
    # 1  final\run_1\*.csv
    # ..

    # Split the full path into directory and filename
    files_split = files['fullpath'].str.rsplit("/", 1, expand=True).rename(columns={0: 'path', 1:'filename'})

    #    path       filename
    # 0  final\run_1  *.csv
    # 1  final\run_1  *.csv
    # ..

    # Join these into one DataFrame
    files = files.join(files_split)

    #    fullpath           path          filename
    # 0  final\run_1\*.csv   final\run_1  *.csv
    # 1  final\run_1\*.csv   final\run_1  *.csv
    # ..

    # Iterate over unique filenames; read CSVs, concat DFs
    avg_losses = pd.DataFrame()
    for fn in files['filename'].unique():
       # fn = files['filename'].unique()[1]
        name = fn.split('.')[0]
        print(name)

        dir, f = split(fn)
        f = f.split('_')
        if any(x in fn for x in ['default', 'ompl']):
            mode = f[-1].split('.')[-2]
        else:
            planner_fn = f[-1].split('.')[-2]
            mode = f[2]  

        paths = files[files['filename'] == fn]['fullpath'] # Get list of fullpaths from unique filenames
        dfs = []

        for path in paths:
            if not any(x in path for x in ['default', 'ompl']):
                df = pd.read_csv(path, index_col=False, sep=',').dropna(axis=1)
                df_resample = resample(df, '2S')
                dfs.append(df_resample) 
                avg = dfs[0]['loss']
                for i in range(len(dfs)):
                    avg = (avg+dfs[i]['loss'])/2

            else:
                df = pd.read_csv(path, index_col=False, sep=',').dropna(axis=1)
                df_resample = resample(df, '2S')
                dfs.append(df_resample) # Get list of dataframes from CSV file paths

                # avg loss
                avg = dfs[0]['t_avg_plan_time']
                for i in range(len(dfs)):
                    avg = (avg+dfs[i]['t_avg_plan_time'])/2

        avg_losses = pd.concat([avg_losses, pd.DataFrame({name :avg})], axis=1)
    return avg_losses



def factor_plot(fps, planner_select, planner, modes=None):
    fps = sorted(fps.copy())
    plt.rcParams['figure.figsize'] = [15, 10]
    fig,ax = plt.subplots(1)
    cm = plt.get_cmap('jet')
    ax.set_prop_cycle('color', [cm(i) for i in np.linspace(0, 1, 10)])
    dir, fn = split(fps[0])
    
    elapsed_time = np.linspace(0, 7200, 500)
    default_fn = planner_select+'_default.csv'
    default_df = pd.read_csv(join(dir, default_fn), index_col=False)
    default = float(default_df[(default_df['planner'].str.contains(planner))]['t_avg_plan_time'])
    default_df = pd.DataFrame(
        {'elapsed_time': elapsed_time, 'default': default})
    default_df['default'] = default/default_df['default']
    ax.plot('elapsed_time', 'default', data=default_df, label='default')

    ompl_fn = planner_select+'_ompl.csv'
    ompl_df = pd.read_csv(join(dir, ompl_fn), index_col=False)
    ompl = float(ompl_df[(ompl_df['planner'].str.contains(planner))]['t_avg_plan_time'])
    ompl_df = pd.DataFrame({'elapsed_time': elapsed_time, 'ompl': ompl})
    ompl_df['ompl'] = default/ompl_df['ompl']
    ax.plot('elapsed_time', 'ompl', data=ompl_df, label='ompl')

    # Planner select filepaths
    for fp in fps:
        if planner_select in fp:
            if not any(x in fp for x in ['default', 'ompl']):
                # f = <planner_select>_<mode>_<planner>.csv
                dir, fn = split(fp)
                fn = fn.split('_')
                planner_fn = fn[-1].split('.')[-2]
                mode = fn[2]
                
                if (modes != None) and (mode not in modes):
                    fps.remove(fp)
                else:
                    # Filtering fns for selected planner
                    if re.search(planner_fn, planner, re.IGNORECASE): 
                        print(fp)
                        mode_df = pd.read_csv(fp, index_col=False)
                        # Remove everything slower than ompl baseline
                        # mode_df.drop(mode_df[mode_df.loss >= ompl].index, inplace=True)
                        max_speed_up = ompl
                        speed_up = list(mode_df['loss'])
                        for idx, loss in enumerate(speed_up):
                            if loss < max_speed_up:
                                max_speed_up = loss
                            else:
                                speed_up[idx] = max_speed_up
                                                                   
                        mode_df = pd.DataFrame({'elapsed_time': mode_df['elapsed_time'], mode: speed_up})
                        mode_df[mode+'_factor'] = ompl/mode_df[mode]
                        ax.plot('elapsed_time', mode+'_factor', data=mode_df, label=mode+'_factor')

    
    plt.title(planner_select + ' ' + planner)
    plt.xticks([0, 1440, 2880, 4320, 5760, 7200])
    #                     plt.yticks([0.8, 1.6, 2.4, 3.1, 3.9, 4.7])
    plt.xlabel('Time spent exploring solutions (secs)')
    plt.ylabel('Speed up over default (secs)')
    handles,labels = ax.get_legend_handles_labels()
    ax.legend(handles,labels,loc='upper right')
    plt.show()


In [2]:
avg_losses = get_avg_losses()

132
Cano_etal_ompl
Burger_etal_gp_BKPIECE
Cano_etal_rf_BKPIECE
Burger_etal_rand_BiTRRT
Burger_etal_tpe_BiTRRT
Cano_etal_tpe_BKPIECE
Burger_etal_tpe_BKPIECE
Cano_etal_gp_BKPIECE
Cano_etal_et_BKPIECE
Burger_etal_aucbandit_BiTRRT
Burger_etal_aucbandit_RRTConnect
Burger_etal_et_RRTConnect
Burger_etal_rand_RRTConnect
Burger_etal_aucbandit_BKPIECE
Burger_etal_rf_BiTRRT
Burger_etal_gbrt_BiTRRT
Cano_etal_rand_RRTConnect
Burger_etal_gbrt_BKPIECE
Cano_etal_rf_RRTConnect
Burger_etal_ompl
Burger_etal_gbrt_RRTConnect
Burger_etal_et_BKPIECE
Burger_etal_smac_RRTConnect
Cano_etal_smac_RRTConnect
Burger_etal_gp_RRTConnect
Cano_etal_et_RRTConnect
Cano_etal_smac_BKPIECE
Burger_etal_rf_RRTConnect
Cano_etal_rand_BKPIECE
Burger_etal_et_BiTRRT
Cano_etal_gbrt_RRTConnect
Burger_etal_rand_BKPIECE
Burger_etal_smac_BiTRRT
Burger_etal_default
Cano_etal_gbrt_BKPIECE
Cano_etal_aucbandit_BKPIECE
Cano_etal_aucbandit_RRTConnect
Burger_etal_gp_BiTRRT
Burger_etal_tpe_RRTConnect
Burger_etal_rf_BKPIECE
Cano_etal_tpe_RRTCon

In [13]:
# def factor_plot(avg_losses, planner_select, planner, modes=None):
planner_select = 'Burger_etal'
planner = 'BKPIECE'

plt.rcParams['figure.figsize'] = [15, 10]
# fig,ax = plt.subplots(1)
# cm = plt.get_cmap('jet')
# ax.set_prop_cycle('color', [cm(i) for i in np.linspace(0, 1, 10)])
# dir, fn = split(fps[0])

df = avg_losses.copy()
# def_cols = [planner_select+'_default', planner_select+'_ompl']
cols = [c for c in df.columns if planner_select in c if planner in c]
cols = sorted(cols)
cols = [planner_select+'_ompl'] + cols
cols = [planner_select+'_default'] + cols
ps_df = df[cols]

legend = [c.split('_')[2] for c in cols]
ps_df.columns = legend

ompl = float(ps_df['ompl'][0])

default = float(ps_df['default'][0])

speedup_df = pd.DataFrame({'default': default/ps_df['default'], 'ompl': default/ps_df['ompl']})

max_speedup = ompl
# def speedup(x):
#     if x < max_speedup:
#         max_speedup = x
#     else:
#         x = max_speedup
#     x = ompl/x
#     return x
 

# speedup_df.plot(colormap='jet')

In [14]:
ps_df.head()

,default,ompl,aucbandit,et,gbrt,gp,rand,rf,smac,tpe
elapsed_time,,,,,,,,,,
00:00:00,0.720435,1.777359,3.000000,3.000000,2.853844,2.772393,2.931013,2.386760,0.728136,1.553238
00:00:02,0.720435,1.777359,3.000000,3.000000,2.853844,2.772393,2.931013,2.386760,0.897452,1.553238
00:00:04,0.720435,1.777359,3.000000,3.000000,2.853844,2.743494,2.931013,2.328151,1.066769,1.553238
00:00:06,0.720435,1.777359,3.000000,3.000000,2.853844,2.685697,2.944810,2.386144,1.236085,1.553238
00:00:08,0.720435,1.777359,3.002182,2.982376,2.853844,2.627900,2.972405,2.444137,1.405402,1.642666


In [22]:
speedup = list(ps_df['rand'])

In [27]:
max_speedup = ompl
for idx, loss in enumerate(speedup):
    if loss < max_speedup:
        max_speedup = loss
    else:
        speedup[idx] = max_speedup
speedup
    

[1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.7773589968681336,
 1.764944632084325,
 1.4002310424354376,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.280800826976881,
 1.28

In [32]:
mode_df = pd.Series(speedup, index=ps_df['rand'].index, name='rand')
mode_df

elapsed_time
00:00:00    1.777359
00:00:02    1.777359
00:00:04    1.777359
00:00:06    1.777359
00:00:08    1.777359
00:00:10    1.777359
00:00:12    1.777359
00:00:14    1.777359
00:00:16    1.777359
00:00:18    1.777359
00:00:20    1.777359
00:00:22    1.777359
00:00:24    1.777359
00:00:26    1.777359
00:00:28    1.777359
00:00:30    1.777359
00:00:32    1.777359
00:00:34    1.777359
00:00:36    1.777359
00:00:38    1.777359
00:00:40    1.777359
00:00:42    1.777359
00:00:44    1.777359
00:00:46    1.777359
00:00:48    1.777359
00:00:50    1.777359
00:00:52    1.777359
00:00:54    1.777359
00:00:56    1.777359
00:00:58    1.777359
              ...   
01:59:00    1.173064
01:59:02    1.173064
01:59:04    1.173064
01:59:06    1.173064
01:59:08    1.173064
01:59:10    1.173064
01:59:12    1.173064
01:59:14    1.173064
01:59:16    1.173064
01:59:18    1.173064
01:59:20    1.173064
01:59:22    1.173064
01:59:24    1.173064
01:59:26    1.173064
01:59:28    1.173064
01:59:30    1.173064


In [33]:
mode_df = ompl/mode_df
mode_df

elapsed_time
00:00:00    1.000000
00:00:02    1.000000
00:00:04    1.000000
00:00:06    1.000000
00:00:08    1.000000
00:00:10    1.000000
00:00:12    1.000000
00:00:14    1.000000
00:00:16    1.000000
00:00:18    1.000000
00:00:20    1.000000
00:00:22    1.000000
00:00:24    1.000000
00:00:26    1.000000
00:00:28    1.000000
00:00:30    1.000000
00:00:32    1.000000
00:00:34    1.000000
00:00:36    1.000000
00:00:38    1.000000
00:00:40    1.000000
00:00:42    1.000000
00:00:44    1.000000
00:00:46    1.000000
00:00:48    1.000000
00:00:50    1.000000
00:00:52    1.000000
00:00:54    1.000000
00:00:56    1.000000
00:00:58    1.000000
              ...   
01:59:00    1.515143
01:59:02    1.515143
01:59:04    1.515143
01:59:06    1.515143
01:59:08    1.515143
01:59:10    1.515143
01:59:12    1.515143
01:59:14    1.515143
01:59:16    1.515143
01:59:18    1.515143
01:59:20    1.515143
01:59:22    1.515143
01:59:24    1.515143
01:59:26    1.515143
01:59:28    1.515143
01:59:30    1.515143


In [36]:
modes = legend
modes = ['rand']
for mode in modes:
    if not any(x in mode for x in ['default', 'ompl']):

        #         if (modes != None) and (mode not in modes):
        #             fps.remove(fp)
        #         else:
        #         max_speedup = ompl
        speedup = list(ps_df[mode])

        for idx, loss in enumerate(speedup):
            if loss < max_speedup:
                max_speedup = loss
            else:
                speedup[idx] = max_speedup

        mode_df = pd.Series(speedup, index=ps_df[mode].index, name=mode)
        mode_df = default/mode_df
        speedup_df = pd.concat([speedup_df, mode_df], axis=1)
        #speedup_df[mode] = speedup_df[mode].apply(lambda x : ompl/x)
#                 mode_df = pd.DataFrame({'elapsed_time': mode_df['elapsed_time'], mode: speedup})
#                 mode_df[mode+'_factor'] = ompl/mode_df[mode]
#                 ax.plot('elapsed_time', mode+'_factor', data=mode_df, label=mode+'_factor')

# speedup_df = pd.concat([speedup_df, pd.DataFrame({mode:})])
speedup_df.head()

,default,ompl,rand,rand
elapsed_time,,,,
00:00:00,1.0,0.40534,1.515143,0.614148
00:00:02,1.0,0.40534,1.515143,0.614148
00:00:04,1.0,0.40534,1.515143,0.614148
00:00:06,1.0,0.40534,1.515143,0.614148
00:00:08,1.0,0.40534,1.515143,0.614148


In [ ]:
speedup_df.plot(colormap='jet')

In [ ]:
print(results_fps_2[0])
# for f in results_fps_2:
df = pd.read_csv(results_fps_2[0])
resampled = resample(df, '2S')
resampled.head()

print(results_fps_1[0])
# for f in results_fps_2:
df1 = pd.read_csv(results_fps_1[0])
resampled1 = resample(df1, '2S')
resampled1.head()

In [ ]:
# resampled1['loss'].plot()
resampled['loss'].plot()

In [ ]:
# x = len(list(df['n_trials'].cols.values))
average = (resampled['loss']+resampled1['loss'])/2
resampled['loss'].head()
resampled1['loss'].head()
average.head()
# for i in range(len(x)):
#     pass


## Speedup Factor Plots

In [ ]:

results_fps_1 = [file for file in glob.glob(join('final', 'run_1', '*.csv'))]
print(len(results_fps_1))

results_fps_2 = [file for file in glob.glob(join('final', 'run_2', '*.csv'))]
print(len(results_fps_2))

results_fps_3 = [file for file in glob.glob(join('final', 'run_3', '*.csv'))]
print(len(results_fps_3))

### Cano_etal BKPIECE

In [ ]:
factor_plot(results_fps_1, 'Cano_etal', 'BKPIECE')

In [ ]:
factor_plot(results_fps_2, 'Cano_etal', 'BKPIECE')

In [ ]:
factor_plot(results_fps_3, 'Cano_etal', 'BKPIECE')

### Cano_etal RRTConnect


In [ ]:
factor_plot(results_fps_1, 'Cano_etal', 'RRTConnect')

In [ ]:
factor_plot(results_fps_2, 'Cano_etal', 'RRTConnect')

In [ ]:
factor_plot(results_fps_3, 'Cano_etal', 'RRTConnect')

### Burger_etal BKPIECE

In [ ]:
factor_plot(results_fps_1, 'Burger_etal', 'BKPIECE')

In [ ]:
factor_plot(results_fps_2, 'Burger_etal', 'BKPIECE')

In [ ]:
factor_plot(results_fps_3, 'Burger_etal', 'BKPIECE')

### Burger_etal RRTConnect

In [ ]:
factor_plot(results_fps_1, 'Burger_etal', 'RRTConnect')

In [ ]:
factor_plot(results_fps_2, 'Burger_etal', 'RRTConnect')

In [ ]:
factor_plot(results_fps_3, 'Burger_etal', 'RRTConnect')

### Burger_etal BiTRRT

In [ ]:
factor_plot(results_fps_1, 'Burger_etal', 'BiTRRT')

In [ ]:
factor_plot(results_fps_2, 'Burger_etal', 'BiTRRT')

In [ ]:
factor_plot(results_fps_3, 'Burger_etal', 'BiTRRT')